In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from torch.autograd import Variable
import torch.onnx
import _init_paths

import os

import easydict

import torch
import torch.utils.data
from opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory

In [2]:
opt = easydict.EasyDict({"K": 100,
"aggr_weight": 0.0,
"agnostic_ex": False,
"arch": "dla_34",
"aug_ddd": 0.5,
"aug_rot": 0,
"batch_size": 16,
"cat_spec_wh": False,
"center_thresh": 0.1,
"chunk_sizes": [15],
"data_dir": "/mnt/nas/data",
"dataset": "mobiltech",
"debug": 0,
"debug_dir": "/mnt/nas/centernet/exp/ctdet/mobiltech/mobiltech_dla/debug",
"debugger_theme": "white",
"demo": "/root/practice/CenterNet/images/mobiltech_test",
"dense_hp": False,
"dense_wh": False,
"dep_weight": 1,
"dim_weight": 1,
"down_ratio": 4,
"eval_oracle_dep": False,
"eval_oracle_hm": False,
"eval_oracle_hmhp": False,
"eval_oracle_hp_offset": False,
"eval_oracle_kps": False,
"eval_oracle_offset": False,
"eval_oracle_wh": False,
"exp_dir": "/mnt/nas/centernet/exp/ctdet",
"exp_id": "mobiltech_dla",
"fix_res": True,
"flip": 1.5,
"flip_test": True,
"gpus": [0],
"gpus_str": 0,
"head_conv": 256,
"heads": {'hm': 66, 'wh': 2, 'reg': 2},
"hide_data_time": False,
"hm_hp": True,
"hm_hp_weight": 1,
"hm_weight": 1,
"hp_weight": 1,
"input_h": 512,
"input_res": 512,
"input_w": 512,
"keep_res": False,
"kitti_split": "3dop",
"load_model": "/mnt/nas/centernet/exp/ctdet/mobiltech/mobiltech_dla/model_230.pth",
"lr": 0.0005,
"lr_step": [180, 210],
"master_batch_size": 15,
"mean": [[[0.40789655, 0.44719303, 0.47026116]]],
"metric": "loss",
"mse_loss": False,
"nms": False,
"no_color_aug": False,
"norm_wh": False,
"not_cuda_benchmark": False,
"not_hm_hp": False,
"not_prefetch_test": False,
"not_rand_crop": False,
"not_reg_bbox": False,
"not_reg_hp_offset": False,
"not_reg_offset": False,
"num_classes": 66,
"num_epochs": 230,
"num_iters": -1,
"num_stacks": 1,
"num_workers": 4,
"off_weight": 1,
"output_h": 128,
"output_res": 128,
"output_w": 128,
"pad": 31,
"peak_thresh": 0.2,
"print_iter": 0,
"rect_mask": False,
"reg_bbox": True,
"reg_hp_offset": True,
"reg_loss": "l1",
"reg_offset": True,
"resume": False,
"root_dir": "/mnt/nas",
"rot_weight": 1,
"rotate": 0,
"save_all": True,
"save_dir": "/mnt/nas/centernet/exp/ctdet/mobiltech/mobiltech_dla",
"scale": 0.4,
"scores_thresh": 0.1,
"seed": 317,
"shift": 0.1,
"std": [[[0.2886383, 0.27408165, 0.27809834]]],
"task": "ctdet",
"test": False,
"test_scales": [1.0],
"trainval": False,
"val_intervals": 5,
"vis_thresh": 0.3,
"wh_weight": 0.1})

In [3]:
torch.manual_seed(opt.seed)
torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test
Dataset = get_dataset(opt.dataset, opt.task)
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
print(opt)

heads {'hm': 66, 'wh': 2, 'reg': 2}
{'K': 100, 'aggr_weight': 0.0, 'agnostic_ex': False, 'arch': 'dla_34', 'aug_ddd': 0.5, 'aug_rot': 0, 'batch_size': 16, 'cat_spec_wh': False, 'center_thresh': 0.1, 'chunk_sizes': [15], 'data_dir': '/mnt/nas/data', 'dataset': 'mobiltech', 'debug': 0, 'debug_dir': '/mnt/nas/centernet/exp/ctdet/mobiltech/mobiltech_dla/debug', 'debugger_theme': 'white', 'demo': '/root/practice/CenterNet/images/mobiltech_test', 'dense_hp': False, 'dense_wh': False, 'dep_weight': 1, 'dim_weight': 1, 'down_ratio': 4, 'eval_oracle_dep': False, 'eval_oracle_hm': False, 'eval_oracle_hmhp': False, 'eval_oracle_hp_offset': False, 'eval_oracle_kps': False, 'eval_oracle_offset': False, 'eval_oracle_wh': False, 'exp_dir': '/mnt/nas/centernet/exp/ctdet', 'exp_id': 'mobiltech_dla', 'fix_res': True, 'flip': 1.5, 'flip_test': True, 'gpus': [0], 'gpus_str': 0, 'head_conv': 256, 'heads': {'hm': 66, 'wh': 2, 'reg': 2}, 'hide_data_time': False, 'hm_hp': True, 'hm_hp_weight': 1, 'hm_weight':

In [4]:

model = create_model(opt.arch, opt.heads, opt.head_conv)
optimizer = torch.optim.Adam(model.parameters(), opt.lr)
models= load_model(model, opt.load_model)
models= model.cuda()
dummy_input = torch.randn(16,3,512,512,device='cuda')

loaded /mnt/nas/centernet/exp/ctdet/mobiltech/mobiltech_dla/model_230.pth, epoch 230


In [5]:

torch.onnx.export(models, dummy_input, "centernet.onnx", verbose=True)

ValueError: Auto nesting doesn't know how to process an input object of type int. Accepted types: Tensors, or lists/tuples of them

In [ ]:
25
16
16
16
16
16
1
7
14
21
14
9



20
텐서
14
텐서
9
텐서
5
텐서
14
텐서
5
텐서
14
텐서
텐서
텐서
9
텐서
5
텐서
14
텐서
5
텐서
14
텐서
텐서
텐서
9
텐서
14
텐서
텐서
3
리스트
리스트
텐서
텐서
텐서
텐서
텐서
텐서
리스트
리스트
텐서
텐서
텐서
텐서
텐서
텐서
히어
4
